In [1]:
import torch
import pytorch_lightning as pl
import os, os.path
import fnmatch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader 
import torch.nn.functional as F
import torch.nn as nn
from PIL import Image
from torch.nn.init import kaiming_normal_, constant_
import cv2
import numpy as np
import matplotlib.pyplot as plt
import flowiz
from pytorch_lightning.strategies import DDPStrategy

import flow_transforms

/home/fi/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/fi/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/fi/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/fi/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## utilities 

## dataset

In [2]:
class DataModuleFlyingChairs(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
          
        # Directory to store FlyingChairs Data
        self.download_dir = './root'     
        # Defining batch size of our data
        self.batch_size =  8 if torch.cuda.is_available() else 32
          
        # Defining transforms to be applied on the data
        self.div_flow = 20 #Factor by which we divide the output (thus >=1). It makes training more stable to deal with low numbers than big ones.
                      #Normalized for the Flying Chair Dataset (https://github.com/ClementPinard/FlowNetPytorch/issues/101#issuecomment-805222823)
        
        
        self.input_transform = transforms.Compose([flow_transforms.ArrayToTensor(), 
                                      transforms.Normalize(mean=[0,0,0], std=[255,255,255]), 
                                      transforms.Normalize(mean=[0.45,0.432,0.411], std=[1,1,1])
                                     ])

        self.target_transform = transforms.Compose([flow_transforms.ArrayToTensor(),
                                       transforms.Normalize(mean=[0,0],std=[self.div_flow,self.div_flow])])

        self.co_transform = flow_transforms.Compose([flow_transforms.RandomCrop((320,448)), 
                                        flow_transforms.RandomVerticalFlip(),
                                        flow_transforms.RandomHorizontalFlip()])
        
    def get_input_transform():
        return  self.input_transform
    
    def get_target_transform():
        return  self.target_transform 
    
    def get_co_transform():
        return  self.co_transform
    
    def train_transformation(self, input1, input2,flow,valid_flow_mask):
        
        valid_flow_mask= None
        input1 = np.array(input1,dtype= np.float32)
        input2 = np.array(input2,dtype= np.float32)
        #print(f"array start f1 shape {input1.shape}")
        
        input1   = np.transpose (input1,(2,0,1))
        input2   = np.transpose (input2,(2,0,1))
        #print(f"array  f1 shape {flow.shape}")
        #print(f"array  n1 shape {input1.shape}")
        #print(f"array  n2 shape {input2.shape}")
      #  inputs = [input1,input2]
        
        
#         inputs, flow = self.co_transform(inputs, flow)

      
#         inputs[0] = self.input_transform(inputs[0])
#         inputs[1] = self.input_transform(inputs[1])
        
#         flow = self.target_transform(flow)
        
   #     input1, input2 = inputs
        return input1, input2 , flow, valid_flow_mask

    def test_transformation(self, input1, input2,flow,valid_flow_mask):
        
        valid_flow_mask= None
        input1= np.array(input1)
        input2= np.array(input2)
        flow   = np.transpose(flow)
        inputs = [input1,input2]
        
                
        crop=flow_transforms.CenterCrop((370,1224))
        
        inputs, flow = crop(inputs, flow)
     
        inputs[0] = self.input_transform(inputs[0])
        inputs[1] = self.input_transform(inputs[1])
        
        flow = self.target_transform(flow)
        
        input1, input2 = inputs
        return input1, input2 , flow ,valid_flow_mask

    def prepare_data(self):
        pass
       
  
    def setup(self, stage=None):
        
          # Loading our data after applying the transforms
        self.train_data = datasets.FlyingChairs(self.download_dir,
                              split = "train", 
                              transforms = self.train_transformation)

        self.test_data = datasets.FlyingChairs(self.download_dir,
                                        split = "val",
                                        transforms = self.test_transformation)
  
    def train_dataloader(self):
        
          # Generating train_dataloader
        return DataLoader(self.train_data, 
                          batch_size = self.batch_size,
                          num_workers = 8)
  
   
  
    def test_dataloader(self):
        
        # Generating test_dataloader
        return DataLoader(self.test_data,
                          batch_size = self.batch_size,
                          num_workers = 100)

In [3]:
dataModulo = DataModuleFlyingChairs()

In [4]:
a= datasets.FlyingChairs('./root' ,
                              split = "train", 
                              transforms = dataModulo.train_transformation)

In [5]:


# #a3=a2[1].cpu().detach().numpy()
# #a2[0].shape
# #a3=np.transpose (a3,(1,2a,0))
# print(type(a[1]))
# plt.imshow(a[1][1])

# type(a2[0][1][20])


In [6]:
dataModulo.setup()
b=dataModulo.train_dataloader()

In [7]:
#for b1, b2 in enumerate(b):
   #print(b2[0].shape)

## model basic blocks

In [8]:
def convBlock( in_channels, out_channels, kernel_size=3, stride=1, batchNormalization=False):
    if(batchNormalization== True):
        return nn.Sequential( nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                                        stride=stride,  padding=(kernel_size-1)//2, bias=False),
                              nn.BatchNorm2d(out_channels),
                              nn.LeakyReLU(negative_slope=0.1,inplace=True)
                            )
    else:
        return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                                        stride=stride,  padding=(kernel_size-1)//2, bias=True),
                              nn.LeakyReLU(negative_slope=0.1,inplace=True)
                            )
    
def predict_block(in_channels):
    return nn.Conv2d(in_channels, 2, kernel_size=3, stride=1, padding =1, bias=False)

def deconvolution_block(in_channels, out_channels):
    return nn.Sequential ( nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
                          nn.LeakyReLU(negative_slope=0.1,inplace=True)
                        )

#Define a Cropping Operation
def crop_like(input, target):
    if input.size()[2:] == target.size()[2:]:
        return input
    else:
        return input[:, :, :target.size(2), :target.size(3)]


## model 

In [9]:
class FlowNetS(pl.LightningModule):

    def __init__(self):
        super().__init__()
 
        #ENCODER PART
        #self.batchNorm = batchNorm
        self.conv1   = convBlock(6,   64, kernel_size=7, stride=2, batchNormalization=True)
        self.conv2   = convBlock(64,  128, kernel_size=5, stride=2, batchNormalization=True)
        self.conv3   = convBlock(128,  256, kernel_size=5, stride=2, batchNormalization=True)
        self.conv3_1 = convBlock(256,  256, batchNormalization=True)
        self.conv4   = convBlock(256,  512, stride=2, batchNormalization=True)
        self.conv4_1 = convBlock(512,  512, batchNormalization=True)
        self.conv5   = convBlock( 512,  512, stride=2, batchNormalization=True)
        self.conv5_1 = convBlock(512,  512, batchNormalization=True)
        self.conv6   = convBlock(512, 1024, stride=2, batchNormalization=True)
        self.conv6_1 = convBlock(1024, 1024, batchNormalization=True) # Note: This one doesn't exist in the paper, but it does in their implementation

        #REFINEMENT PART
        self.deconv5 = deconvolution_block(1024,512)
        self.deconv4 = deconvolution_block(1026,256)
        self.deconv3 = deconvolution_block(770,128)
        self.deconv2 = deconvolution_block(386,64)

        self.predict_flow6 = predict_block(1024)
        self.predict_flow5 = predict_block(1026)
        self.predict_flow4 = predict_block(770)
        self.predict_flow3 = predict_block(386)
        self.predict_flow2 = predict_block(194)

        self.upsampled_flow6_to_5 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)
        self.upsampled_flow5_to_4 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)
        self.upsampled_flow4_to_3 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)
        self.upsampled_flow3_to_2 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)

        
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                # Initialize the Convolutions with "He Initialization" to 0.1 (https://arxiv.org/pdf/1502.01852.pdf)
                kaiming_normal_(m.weight, 0.1)
                if m.bias is not None:
                    # Initialize all bias to 0
                    constant_(m.bias, 0)
            # Initialize the BatchNorm Convolutions with "He Initialization" to 1 (https://arxiv.org/pdf/1502.01852.pdf)
            elif isinstance(m, nn.BatchNorm2d):
                constant_(m.weight, 1)
                constant_(m.bias, 0)
                
    def forward (self, x):
        # ENCODER
        out_conv2 = self.conv2(self.conv1(x))
        out_conv3 = self.conv3_1(self.conv3(out_conv2))
        out_conv4 = self.conv4_1(self.conv4(out_conv3))
        out_conv5 = self.conv5_1(self.conv5(out_conv4))
        out_conv6 = self.conv6_1(self.conv6(out_conv5))
        
        # REFINEMENT
        flow6       = self.predict_flow6(out_conv6)
        flow6_up    = crop_like(self.upsampled_flow6_to_5(flow6), out_conv5)
        out_deconv5 = crop_like(self.deconv5(out_conv6), out_conv5)

        concat5 = torch.cat((out_conv5,out_deconv5,flow6_up),1)
        flow5       = self.predict_flow5(concat5)
        flow5_up    = crop_like(self.upsampled_flow5_to_4(flow5), out_conv4)
        out_deconv4 = crop_like(self.deconv4(concat5), out_conv4)

        concat4 = torch.cat((out_conv4,out_deconv4,flow5_up),1)
        flow4       = self.predict_flow4(concat4)
        flow4_up    = crop_like(self.upsampled_flow4_to_3(flow4), out_conv3)
        out_deconv3 = crop_like(self.deconv3(concat4), out_conv3)

        concat3 = torch.cat((out_conv3,out_deconv3,flow4_up),1)
        flow3       = self.predict_flow3(concat3)
        flow3_up    = crop_like(self.upsampled_flow3_to_2(flow3), out_conv2)
        out_deconv2 = crop_like(self.deconv2(concat3), out_conv2)

        concat2 = torch.cat((out_conv2,out_deconv2,flow3_up),1)
        flow2 = self.predict_flow2(concat2)

        if self.training:
            return flow2,flow3,flow4,flow5,flow6
        else:
            return flow2
        
    def weight_parameters(self):
        return [param for name, param in self.named_parameters() if 'weight' in name]

    def bias_parameters(self):
        return [param for name, param in self.named_parameters() if 'bias' in name]
      
        
    def training_step(self, batch, batch_idx):
        # --------------------------
        # REPLACE WITH YOUR OWN
        batch_time = AverageMeter()
        data_time = AverageMeter()
        losses = AverageMeter()
        flow2_EPEs = AverageMeter()
        multiscale_weights = [0.005,0.01,0.02,0.08,0.32] # from output_flow to flow6
        
        #print (f"batch len {len(batch)}")
        #print (f"batch first element shape {batch[0].shape}")
        #print (f"batch second element shape {batch[1].shape}")
        #print (f"batch tird element shape {batch[2].shape}")
        x,x1, y = batch
        x_train = torch.cat([x,x1],1)
        #print (f"x concat  shape {x_train.shape}")
        #print (f"x concat type{type(x_train)}")
        output=self(x_train)
        h, w = y.size()[-2:]
        output = [F.interpolate(output[0], (h,w)), *output[1:]]
        
        
        
        
        #print (f"x_hat type{type(output)}")
        #print (f"x_hat len  {len(output)}")
        #print (f"x_hat[0] shape type{output[0].shape}")
        loss = multiscaleEPE(output, y, weights=multiscale_weights, sparse=True)

        # Compute the Output EPE
        flow2_EPE = 20 * realEPE(output[0], y, sparse=True)  #div_flow

        # Record loss and EPE
        losses.update(loss.item(), y.size(0))
        #train_writer.add_scalar('train_loss', loss.item(), n_iter)
        flow2_EPEs.update(flow2_EPE.item(), y.size(0))
        self.log('train_loss', loss)
        return loss
        # --------------------------



    def test_step(self, batch, batch_idx):
     # --------------------------list all  numbers from 1 to 100 python
        # REPLACE WITH YOUR OWN
  
        x, y = batch
        x_hat = self(x)
    
        loss = F.mse_loss(x_hat, x)
        self.log('train_loss', loss)
        return loss
        # --------------------------

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=10e-4)
        return optimizer

## Loss function

In [10]:
def EPE(input_flow, target_flow, sparse=False, mean=True):
    EPE_map = torch.norm(target_flow-input_flow,2,1)
    batch_size = EPE_map.size(0)
    if sparse:
        # invalid flow is defined with both flow coordinates to be exactly 0
        mask = (target_flow[:,0] == 0) & (target_flow[:,1] == 0)
        EPE_map = EPE_map[~mask]
    if mean:
        return EPE_map.mean()
    else:
        return EPE_map.sum()/batch_size
    
def realEPE(output, target, sparse=False):
    b, _, h, w = target.size()
    upsampled_output = F.interpolate(output, (h,w), mode='bilinear', align_corners=False) # used to resize the output
    return EPE(upsampled_output, target, sparse, mean=True)



def sparse_max_pool(input, size):
    '''Downsample the input by considering 0 values as invalid.
    Unfortunately, no generic interpolation mode can resize a sparse map correctly,
    the strategy here is to use max pooling for positive values and "min pooling"
    for negative values, the two results are then summed.
    This technique allows sparsity to be minized, contrary to nearest interpolation,
    which could potentially lose information for isolated data points.'''

    positive = (input > 0).float()
    negative = (input < 0).float()
    output = F.adaptive_max_pool2d(input * positive, size) - F.adaptive_max_pool2d(-input * negative, size)
    return output


def multiscaleEPE(network_output, target_flow, weights=None, sparse=False):
    def one_scale(output, target, sparse):

        b, _, h, w = output.size()
        if sparse:
            target_scaled = sparse_max_pool(target, (h, w))
        else:
            target_scaled = F.interpolate(target, (h, w), mode='area')
        return EPE(output, target_scaled, sparse, mean=False)
    

    if type(network_output) not in [tuple, list]:
        network_output = [network_output]
    if weights is None:
        weights = [0.005, 0.01, 0.02, 0.08, 0.32]  # as in original article
    assert(len(weights) == len(network_output))

    loss = 0
    for output, weight in zip(network_output, weights):
        loss += weight * one_scale(output, target_flow, sparse)
    return loss




class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __repr__(self):
        return '{:.3f} ({:.3f})'.format(self.val, self.avg)



## training

In [1]:
data=DataModuleFlyingChairs()
model = FlowNetS()



if __name__ == "__main__": 
     ddp= DDPStrategy(find_unused_parameters=False)
     trainer = pl.Trainer(gpus=2, max_epochs=3, progress_bar_refresh_rate=20,strategy= ddp)
     trainer.fit(model,data)

# trainer = pl.Trainer(gpus=1, max_epochs=4, progress_bar_refresh_rate=20)
# trainer.fit(model,data)

NameError: name 'DataModuleFlyingChairs' is not defined